# Citibike - Statistical Analysis Project

August 1, 2020

<u>About the Data:</u>   https://www.citibikenyc.com/system-data <br>
<u>Data Source:</u>    https://s3.amazonaws.com/tripdata/index.html<br>
<u>Data Date Range:</u>    June 2019 - June 2020

## About the Data

<u>Data Columns:</u>
- Trip Duration (seconds)
- Start Time and Date
- Stop Time and Date
- Start Station Name
- End Station Name
- Station ID
- Station Lat/Long
- Bike ID
- User Type (Customer = 24-hour pass or 3-day pass user; Subscriber = Annual Member)
- Gender (Zero=unknown; 1=male; 2=female)
- Year of Birth

<u>Additional Notes:</u>
- Test trips & trips < 60 seconds (potentially false starts or users tyring to redock bike) have been removed.
- Milage estimates are calculated using an assumed speed of 7.456 miles per hour, up to two hours. Trips over two hours max-out at 14.9 miles. Once you opt into Ride Insights, the Citi Bike app will use your phone's location to record the route you take between your starting and ending Citi Bike station to give exact mileage.
- We only include trips that begin at publicly available stations (thereby excluding trips that originate at our depots for rebalancing or maintenance purposes).

## Setup

### Import libraries

In [1]:
# import relevant libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import pickle
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option('display.float_format', lambda x: '%f' % x)

C:\Users\Grace\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Import data

In [2]:
os.getcwd()

'C:\\Users\\Grace\\Documents\\My_Git_Repos\\citibike-analysis'

In [3]:
os.chdir("data/2019") 

In [4]:
os.listdir()

['201901-citibike-tripdata.csv',
 '201902-citibike-tripdata.csv',
 '201903-citibike-tripdata.csv',
 '201904-citibike-tripdata.csv',
 '201905-citibike-tripdata.csv',
 '201906-citibike-tripdata.csv']

In [5]:
# import all 2019 files into a list of dataframes
dfs_2019 = []
for file in os.listdir():
    dfs_2019.append(pd.read_csv(file))

In [6]:
# concatenate all dataframes into a single dataframe
df_2019 =pd.concat(dfs_2019,ignore_index=True)

In [7]:
df_2019.shape

(9055018, 15)

In [8]:
# pickle the dataframe for easy re-import
df_2019.to_pickle('df_2019.pickle')

In [13]:
os.chdir("..") 

In [15]:
os.getcwd()

'C:\\Users\\Grace\\Documents\\My_Git_Repos\\citibike-analysis\\data'

In [16]:
os.chdir('2020')

In [17]:
os.listdir()

['202001-citibike-tripdata.csv',
 '202002-citibike-tripdata.csv',
 '202003-citibike-tripdata.csv',
 '202004-citibike-tripdata.csv',
 '202005-citibike-tripdata.csv',
 '202006-citibike-tripdata.csv']

In [18]:
# import all 2020 files into a list of dataframes
dfs_2020 = []
for file in os.listdir():
    dfs_2020.append(pd.read_csv(file))

In [19]:
# concatenate all dataframes into a single dataframe
df_2020 =pd.concat(dfs_2020,ignore_index=True)

In [20]:
df_2020.shape

(7508808, 15)

In [21]:
# pickle the dataframe for easy re-import
df_2020.to_pickle('df_2020.pickle')

## Data QA

### Set datatypes

Check that you have the right datatypes.

In [22]:
# original datatypes
df_2019.dtypes

tripduration                 int64
starttime                   object
stoptime                    object
start station id           float64
start station name          object
start station latitude     float64
start station longitude    float64
end station id             float64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
birth year                   int64
gender                       int64
dtype: object

In [23]:
df_2020.dtypes

tripduration                 int64
starttime                   object
stoptime                    object
start station id             int64
start station name          object
start station latitude     float64
start station longitude    float64
end station id               int64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
birth year                   int64
gender                       int64
dtype: object

`starttime` and `stoptime` are current set as strings so converting them to datetime.

In [24]:
df_2019['starttime'] = df_2019['starttime'].astype('datetime64')
df_2019['stoptime'] =df_2019['stoptime'].astype('datetime64')

df_2020['starttime'] = df_2020['starttime'].astype('datetime64')
df_2020['stoptime'] =df_2020['stoptime'].astype('datetime64')

### Create features


<i>New features:</i>
1. `tripduration_min`
2. `starttime_year`
3. `starttime_month`
4. `age`

Since `tripduration` is in seconds, let's create another column that shows duration in minutes.

In [54]:
df_2019['tripduration_min'] = df_2019['tripduration']/60
df_2020['tripduration_min'] = df_2020['tripduration']/60

It may also be helpful to extract the year & month of each starttimes.

In [56]:
# 2019 data
df_2019['starttime_year'] = pd.DatetimeIndex(df_2019['starttime']).year
df_2019['starttime_month'] = pd.DatetimeIndex(df_2019['starttime']).month

# 2020 data
df_2020['starttime_year'] = pd.DatetimeIndex(df_2020['starttime']).year
df_2020['starttime_month'] = pd.DatetimeIndex(df_2020['starttime']).month

Create `age` from `birth year`.

In [30]:
df_2019['age'] = 2019 - df_2019['birth year']
df_2020['age'] = 2020 - df_2020['birth year']

In [31]:
df_2019[['age', 'birth year']].head()

,age,birth year
0,48,1971
1,55,1964
2,32,1987
3,29,1990
4,40,1979


In [32]:
df_2020[['age', 'birth year']].head()

,age,birth year
0,28,1992
1,51,1969
2,57,1963
3,40,1980
4,38,1982


### Assess summary statistics & look for null values

In [43]:
df_2019.isna().sum()

tripduration                0
starttime                   0
stoptime                    0
start station id           37
start station name         37
start station latitude      0
start station longitude     0
end station id             37
end station name           37
end station latitude        0
end station longitude       0
bikeid                      0
usertype                    0
birth year                  0
gender                      0
age                         0
dtype: int64

Check that the same 37 records are null across the four columns.

In [41]:
df_2019[(df_2019['start station id'].isna()==1) & \
        (df_2019['start station name'].isna()==1) &\
       (df_2019['end station id'].isna()==1) & \
       (df_2019['end station name'].isna()==1)].shape

(37, 16)

In [45]:
df_2019.shape

(9055018, 16)

In [46]:
9055018-37

9054981

Drop these 37 rows. Should end up with 9,054,981 (9,055,018-37) rows.

In [47]:
df_2019=df_2019.dropna(how='any')

In [48]:
df_2019.shape

(9054981, 16)

No nulls in the 2020 data.

In [49]:
df_2020.isna().sum()

tripduration               0
starttime                  0
stoptime                   0
start station id           0
start station name         0
start station latitude     0
start station longitude    0
end station id             0
end station name           0
end station latitude       0
end station longitude      0
bikeid                     0
usertype                   0
birth year                 0
gender                     0
age                        0
dtype: int64

Let's look at the summary statics for any odd values.

In [58]:
df_2019.describe()

,tripduration,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,bikeid,birth year,gender,age,tripduration_min,starttime_year,starttime_month
count,9054981.000000,9054981.000000,9054981.000000,9054981.000000,9054981.000000,9054981.000000,9054981.000000,9054981.000000,9054981.000000,9054981.000000,9054981.000000,9054981.000000,9054981.000000,9054981.000000
mean,990.020184,1663.462444,40.737064,-73.982047,1655.304334,40.736844,-73.982251,28278.537632,1979.711640,1.160708,39.288360,16.500336,2019.000000,4.006414
std,11234.735920,1470.033989,0.030657,0.019397,1469.547474,0.030517,0.019466,6622.148641,12.082655,0.527176,12.082655,187.245599,0.000000,1.635945
min,61.000000,72.000000,40.655400,-74.025353,72.000000,40.655400,-74.068601,14529.000000,1857.000000,0.000000,16.000000,1.016667,2019.000000,1.000000
25%,351.000000,383.000000,40.717548,-73.994903,382.000000,40.717488,-73.995299,24872.000000,1969.000000,1.000000,29.000000,5.850000,2019.000000,3.000000
50%,598.000000,515.000000,40.737815,-73.984907,514.000000,40.737262,-73.985624,30189.000000,1982.000000,1.000000,37.000000,9.966667,2019.000000,4.000000
75%,1060.000000,3292.000000,40.757246,-73.971370,3290.000000,40.757148,-73.971789,33753.000000,1990.000000,1.000000,50.000000,17.666667,2019.000000,5.000000
max,3379585.000000,3795.000000,40.814394,-73.907744,3795.000000,40.814394,-73.907744,39934.000000,2003.000000,2.000000,162.000000,56326.416667,2019.000000,6.000000


In [59]:
df_2020.describe()

,tripduration,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,bikeid,birth year,gender,age,tripduration_min,starttime_year,starttime_month
count,7508808.000000,7508808.000000,7508808.000000,7508808.000000,7508808.000000,7508808.000000,7508808.000000,7508808.000000,7508808.000000,7508808.000000,7508808.000000,7508808.000000,7508808.000000,7508808.000000
mean,1296.706479,1909.454409,40.738886,-73.978329,1902.072681,40.738546,-73.978530,33529.882299,1980.496803,1.173623,39.503197,21.611775,2020.000000,3.756091
std,12508.910894,1522.877728,0.033259,0.021976,1523.604046,0.033144,0.022021,8385.868800,12.317975,0.582472,12.317975,208.481848,0.000000,1.838899
min,61.000000,72.000000,40.655400,-74.017134,72.000000,40.655400,-74.071262,14529.000000,1873.000000,0.000000,16.000000,1.016667,2020.000000,1.000000
25%,409.000000,408.000000,40.716250,-73.993934,405.000000,40.716075,-73.994156,28847.000000,1969.000000,1.000000,30.000000,6.816667,2020.000000,2.000000
50%,768.000000,2006.000000,40.739126,-73.981948,2006.000000,40.738274,-73.982079,35323.000000,1983.000000,1.000000,37.000000,12.800000,2020.000000,4.000000
75%,1431.000000,3387.000000,40.761438,-73.963198,3388.000000,40.761149,-73.963198,40419.000000,1990.000000,2.000000,51.000000,23.850000,2020.000000,6.000000
max,3786187.000000,4038.000000,40.842842,-73.897950,4038.000000,40.842842,-73.897950,44542.000000,2004.000000,2.000000,147.000000,63103.116667,2020.000000,6.000000


There are ages (>100) that do not make sense.

There are trip durations that are extremely long (spanning several days).

## Data Analysis

###  Questions

- Ridership numbers
- Duration of trips
- Popular & least popular stations (start & end stations)
- Popular routes (maybe visualize the routes on a map)
- Who's riding Citibikes during COVID (age, gender)?
- How has bike rental behavior changed due to COVID?
- Did we lose subscribers during COVID?

### Ridership Numbers